In [7]:
import pandas as pd
import os
from pathlib import Path

# 设置数据根目录
data_root = Path.home() /  "data" / "rmrb_1948-2024"

def load_rmrb_data(year: int) -> pd.DataFrame:
    """
    加载指定年份的人民日报数据
    
    Args:
        year: 年份 (1946-2024)
        
    Returns:
        包含该年份人民日报文本数据的DataFrame
    """
    if not 1946 <= year <= 2024:
        raise ValueError("年份必须在1946-2024之间")
        
    filename = f"人民日报{year}年文本数据.xlsx"
    file_path = data_root  / filename
    
    if not file_path.exists():
        raise FileNotFoundError(f"找不到{year}年的数据文件: {file_path}")
        
    df = pd.read_excel(file_path)
    return df

# 测试读取1990年的数据
try:
    df_1990 = load_rmrb_data(1990)
    print("数据加载成功!")
    print("\n数据基本信息:")
    print(df_1990.info())
    print("\n前5行数据预览:")
    print(df_1990.head())
except FileNotFoundError as e:
    print(f"错误: {e}")
except Exception as e:
    print(f"发生未知错误: {e}")


数据加载成功!

数据基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31721 entries, 0 to 31720
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年份      31719 non-null  float64
 1   日期      31721 non-null  object 
 2   报纸版次    31720 non-null  object 
 3   标题      31719 non-null  object 
 4   文本内容    31718 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.2+ MB
None

前5行数据预览:
       年份          日期 报纸版次                      标题  \
0  1990.0  1990-01-01  第2版                纺织品喜添新家族   
1  1990.0  1990-01-01  第4版                 祖国的早晨图片   
2  1990.0  1990-01-01  第2版                    精心部署   
3  1990.0  1990-01-01  第1版         中央领导同民主党派党外人士座谈   
4  1990.0  1990-01-01  第1版  江泽民总书记新年前夕接受记者采访谈国内外形势   

                                                文本内容  
0   纺织品喜添新家族 多功能涂层、复合织物在沪亮相 吴复民 1990-01-01 第2版(国内...  
1   祖国的早晨（图片） 刘称奇 1990-01-01 第4版(画刊) 专栏： 祖国的早晨 （年...  
2   精心部署 措施落实 上海广东妥善安排春运 梁兆明 1990-01-01 第2版(国内新闻)...  
3   

In [8]:
from gensim.models import KeyedVectors
from gensim.downloader import base_dir
import os

base_dir

'/home/fangshikai/gensim-data'

In [9]:
# 使用gensim下载预训练的中文词向量
from gensim.models import KeyedVectors
from gensim.downloader import base_dir
import os

# 设置下载目录
DOWNLOAD_DIR = os.path.join(base_dir, "vectors")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# 尝试加载腾讯AI实验室的预训练词向量
# 注意:这个文件较大(约6.7GB),首次下载可能需要一些时间
print("开始下载中文词向量模型...")
try:
    # 检查本地是否已有保存的模型
    local_path = os.path.join(DOWNLOAD_DIR, "chinese_vectors.kv")
    if os.path.exists(local_path):
        print("发现本地已有模型,正在加载...")
        word_vectors = KeyedVectors.load(local_path)
        print("本地模型加载成功!")
    else:
        print("本地未找到模型,开始下载...")
        word_vectors = KeyedVectors.load_word2vec_format(
            'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.vec.gz',
            binary=False
        )
        print("词向量模型下载成功!")
        
        # 保存模型到本地
        word_vectors.save(local_path)
        print(f"模型已保存到: {local_path}")
    
    # 简单测试
    print("\n测试一些词的相似度:")
    print("'法制'和'治理'的相似度:", word_vectors.similarity('法制', '治理'))
    print("\n与'法制'最相似的5个词:")
    print(word_vectors.most_similar('法制', topn=10))
    
except Exception as e:
    print(f"下载或加载词向量时发生错误: {e}")
    print("提示: 您也可以手动下载其他开源的中文词向量，如:")
    print("1. 腾讯词向量: https://ai.tencent.com/ailab/nlp/en/embedding.html")
    print("2. 哈工大词向量: https://github.com/nghuyong/Chinese-Word-Vectors")


开始下载中文词向量模型...
本地未找到模型,开始下载...
词向量模型下载成功!
模型已保存到: /home/fangshikai/gensim-data/vectors/chinese_vectors.kv

测试一些词的相似度:
'法制'和'治理'的相似度: 0.39143848

与'法制'最相似的5个词:
[('法治', 0.7596691250801086), ('網北京', 0.6227442622184753), ('民主', 0.5542482733726501), ('法律', 0.5525763630867004), ('輯刊', 0.5451518893241882), ('司法', 0.5400424003601074), ('普法', 0.5254901051521301), ('成对照', 0.5251033306121826), ('普法网', 0.5246453881263733), ('政法', 0.5194897651672363)]


In [11]:
similar_words = word_vectors.most_similar('法治', topn=50)
print("\n与'法治'最相似的10个词:")
for i, (word, score) in enumerate(similar_words, 1):
    print(f"{i}. {word}: {score:.4f}")


与'法治'最相似的10个词:
1. 法制: 0.7597
2. 法治化: 0.6360
3. 民主: 0.6080
4. 人治: 0.6059
5. 宪政: 0.5964
6. 司法: 0.5771
7. 法律: 0.5632
8. 法治论: 0.5506
9. 司法獨立: 0.5471
10. 法管: 0.5366
11. 法学人: 0.5361
12. 治监: 0.5320
13. 政法: 0.5305
14. 德法兼修: 0.5268
15. 法冶: 0.5251
16. 宪法: 0.5185
17. 反腐: 0.5184
18. 文明: 0.5183
19. 德治: 0.5181
20. 法制化: 0.5153
21. 人权: 0.5145
22. 社会主义: 0.5133
23. 司法化: 0.5115
24. 普法网: 0.5107
25. 普法: 0.5082
26. 限權: 0.5069
27. 道德: 0.5059
28. 有法必依: 0.5052
29. 公权力: 0.5050
30. 行政法: 0.5046
31. 公民: 0.5044
32. 卓泽渊: 0.5030
33. 水治: 0.5015
34. 反腐倡廉: 0.5011
35. 依法: 0.5000
36. 黨的領導: 0.4997
37. 基本法: 0.4993
38. 法援: 0.4981
39. 12•4: 0.4967
40. 治理: 0.4963
41. 法制观: 0.4946
42. 普法办: 0.4945
43. 憲政: 0.4930
44. 弘揚憲: 0.4924
45. 常規療: 0.4886
46. 築牢: 0.4878
47. 检察: 0.4872
48. 法律人: 0.4871
49. 顾大松: 0.4869
50. 宪治: 0.4864


In [ ]:

cosmul_words = word_vectors.most_similar_cosmul('法治', topn=50)
print("\n与'法治' 共现的10个词:")
for i, (word, score) in enumerate(cosmul_words, 1):
    print(f"{i}. {word}: {score:.4f}")


与'法治' 共现的10个词:
1. 法制: 0.8798
2. 法治化: 0.8180
3. 民主: 0.8040
4. 人治: 0.8029
5. 宪政: 0.7982
6. 司法: 0.7885
7. 法律: 0.7816
8. 法治论: 0.7753
9. 司法獨立: 0.7736
10. 法管: 0.7683
11. 法学人: 0.7680
12. 治监: 0.7660
13. 政法: 0.7653
14. 德法兼修: 0.7634
15. 法冶: 0.7626
16. 宪法: 0.7593
17. 反腐: 0.7592
18. 文明: 0.7591
19. 德治: 0.7591
20. 法制化: 0.7577
21. 人权: 0.7573
22. 社会主义: 0.7566
23. 司法化: 0.7557
24. 普法网: 0.7553
25. 普法: 0.7541
26. 限權: 0.7535
27. 道德: 0.7529
28. 有法必依: 0.7526
29. 公权力: 0.7525
30. 行政法: 0.7523
31. 公民: 0.7522
32. 卓泽渊: 0.7515
33. 水治: 0.7508
34. 反腐倡廉: 0.7505
35. 依法: 0.7500
36. 黨的領導: 0.7499
37. 基本法: 0.7497
38. 法援: 0.7490
39. 12•4: 0.7484
40. 治理: 0.7482
41. 法制观: 0.7473
42. 普法办: 0.7473
43. 憲政: 0.7465
44. 弘揚憲: 0.7462
45. 常規療: 0.7443
46. 築牢: 0.7439
47. 检察: 0.7436
48. 法律人: 0.7436
49. 顾大松: 0.7435
50. 宪治: 0.7432


In [14]:
most_similar_result = word_vectors.most_similar(positive=['女人', '国王'], negative=['男人'])
cosmul_result = word_vectors.most_similar_cosmul(positive=['女人', '国王'], negative=['男人'])

print("most_similar 结果:", most_similar_result[:3])
print("most_similar_cosmul 结果:", cosmul_result[:3])

most_similar 结果: [('王后们', 0.6541500091552734), ('王宫', 0.5844749808311462), ('王后', 0.5730476975440979)]
most_similar_cosmul 结果: [('王后们', 0.8665317893028259), ('王宫', 0.8232525587081909), ('王后', 0.8148195147514343)]


In [17]:
most_similar_result = word_vectors.most_similar(positive=['法治', ], negative=['社会主义'])
cosmul_result = word_vectors.most_similar_cosmul(positive=['法治', ], negative=['社会主义'])

print("most_similar 结果:", most_similar_result[:10])
print("most_similar_cosmul 结果:", cosmul_result[:10])

most_similar 结果: [('法管', 0.3306950330734253), ('審查權', 0.3020731210708618), ('人治', 0.2962474524974823), ('法制', 0.295155793428421), ('治', 0.2899714410305023), ('查案', 0.2893400490283966), ('癒', 0.2866268455982208), ('限權', 0.2847197353839874), ('有治', 0.28410476446151733), ('與治', 0.2773081362247467)]
most_similar_cosmul 结果: [('論��', 1.2894666194915771), ('查案', 1.271921992301941), ('法管', 1.2695727348327637), ('有治', 1.2629528045654297), ('審查權', 1.2617477178573608), ('20161113', 1.2613542079925537), ('與治', 1.2588070631027222), ('黃律', 1.2567927837371826), ('癒', 1.2554254531860352), ('治', 1.2547706365585327)]
